This notebook is exactly same with dataset generation step in `inference.ipynb` and `main.ipynb`.

# 0. Prerequisites

## 0-1. Python Libraries

- Albumentations
- opencv-python
- imageio
- numpy
- pandas
- timm
- torch==1.7.0 with cuda toolkit 11.2.2, cudnn8
- pyaml
- adabelief_pytorch
- scikit-learn
- tqdm

In [2]:
import gc
import json
import math
import random
import re
import shutil
import sys
from collections import defaultdict
from dataclasses import dataclass
from multiprocessing import Pool
from os import PathLike
from pathlib import Path
from typing import Any, Tuple, List

import albumentations as A
import cv2
import imageio
import numpy as np
import pandas as pd
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import yaml
from adabelief_pytorch import AdaBelief
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import StratifiedKFold
from timm.models.layers import Conv2dSame
from timm.models.nfnet import ScaledStdConv2dSame
from torch import Tensor
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [3]:
from const.flip import id_flip
from const.label_names import id_to_label, label_names
from utils import AverageMeter, CustomLogger, make_result_dir, seed_everything, tqdm_kwargs

# 1. Dataset

The original dataset is located in `./data/ori`.
And I will make new dataset by cropping the original dataset and it will be located in `./data/crop512_9`.

The new dataset contains both `*.png` and `*.pth` files.
But the `*.pth` files are not used in this code, so you can just ignore it.

In [16]:
dsize = (512, 512)
crop_padding = 120
ratio_limit = 1.2
seq_len = 5

# data numbers where its keypoints contains error
wrong_data = [312, 317, 318, 327, 340, 343, 475, 543, 619, 622, 750, 746]

In [17]:
imagenet_mean = np.array([0.485, 0.456, 0.406]).reshape(1, 1, 3)
imagenet_std = np.array([0.229, 0.224, 0.225]).reshape(1, 1, 3)

In [18]:
out_dir = Path("./data/crop512_9")
if out_dir.exists():
    shutil.rmtree(out_dir)

train_out_dir = out_dir / "train"
test_out_dir = out_dir / "test"
train_out_dir.mkdir(parents=True, exist_ok=True)
test_out_dir.mkdir(parents=True, exist_ok=True)

In [19]:
def elastic_resize(im, bbox, dsize, ratio_limit):
    """resize while keep aspect ratio"""
    # bbox (x1, y1, x2, y2)
    # dsize (w, h)
    # ratio_limit: float

    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]

    if h == w:
        return cv2.resize(im[bbox[1] : bbox[3], bbox[0] : bbox[2]], dsize)

    long = h > w
    a, b = (h, w) if long else (w, h)
    ratio = a / b

    if ratio <= ratio_limit:
        return cv2.resize(im[bbox[1] : bbox[3], bbox[0] : bbox[2]], dsize)

    e, f, g = (bbox[0], bbox[2], im.shape[1]) if long else (bbox[1], bbox[3], im.shape[0])

    db = int(a / ratio_limit)
    c = db - b
    e -= math.ceil(c / 2)
    f += math.floor(c / 2)

    if e < 0:
        f += -e
        e = 0
    elif f > g:
        e -= f - g
        f = g

    e = max(0, e)
    f = min(f, g)

    if long:
        bbox[0], bbox[2] = e, f
    else:
        bbox[1], bbox[3] = e, f
    fb = f - e

    return cv2.resize(im[bbox[1] : bbox[3], bbox[0] : bbox[2]], dsize)

In [20]:
def find_bbox(im, u):
    """
    refer to 게으름뱅이 code share:
        https://dacon.io/competitions/official/235805/codeshare/3373?page=1&dtype=recent
    """
    mask = (im == [255, 0, 0]).all(axis=-1) | (im == [0, 255, 0]).all(axis=-1)

    pos = np.stack(mask.nonzero())
    bbox = np.round(
        np.array(
            (
                np.clip(pos[1, :].min() - u, 0, 1920),
                np.clip(pos[0, :].min() - u, 0, 1920),
                np.clip(pos[1, :].max() + u, 0, 1920),
                np.clip(pos[0, :].max() + u, 0, 1920),
            ),
            dtype=np.float64,
        )
    ).astype(np.int64)

    return bbox

In [21]:
def process_image(impath: Path, keypoints: np.ndarray):
    im = imageio.imread(impath)

    # crop
    u = crop_padding

    if int(impath.parent.name) in wrong_data:
        bbox = find_bbox(im, u)
    else:
        v = keypoints
        bbox = np.round(
            np.array(
                (
                    np.clip(v[:, 0].min() - u, 0, 1920),
                    np.clip(v[:, 1].min() - u, 0, 1080),
                    np.clip(v[:, 0].max() + u, 0, 1920),
                    np.clip(v[:, 1].max() + u, 0, 1080),
                ),
                dtype=np.float32,
            )
        ).astype(np.int64)

    im = elastic_resize(im, bbox, dsize, ratio_limit)

    # standardization
    im2 = (im.astype(np.float32) / 255.0 - imagenet_mean) / imagenet_std
    im2 = torch.from_numpy(im2).permute(2, 0, 1).type(torch.float32)

    return im, im2

## 1-1. Generate Training Dataset

The training image file names are following this format `{dir index}_{image index}_{label index}.png`, e.g. `001_02_003.png`.

In [22]:
def process_dir_train(dirpath: Path):
    with open(dirpath / f"{dirpath.name}.json") as f:
        j = json.load(f)

    diridx = int(dirpath.name)

    label = id_to_label[j["action"][0]]
    label = torch.tensor(label, dtype=torch.long)

    for i, annot in enumerate(j["annotations"]):
        impath = dirpath / f"{i}.png"
        im_org, im = process_image(impath, np.array(annot["data"]))

        # save image
        fname = f"{diridx:03d}_{i:02d}_{label.item():03d}"
        imageio.imwrite(train_out_dir / (fname + ".png"), im_org)
        torch.save(im, train_out_dir / (fname + ".pth"))

In [23]:
dirs = sorted(list(Path("./data/ori/train").glob("*")))

In [24]:
len(dirs)  # 649

649

There are 649 training directories containing multiple images.
I applied parallelism because processing all of these data takes too long.

In [25]:
with Pool() as pool:
    with tqdm(total=len(dirs), ncols=100, file=sys.stdout) as t:
        for _ in pool.imap_unordered(process_dir_train, dirs):
            t.update()

100%|█████████████████████████████████████████████████████████████| 649/649 [12:01<00:00,  1.11s/it]


## 1-2. Generate Test Dataset

The test image file names are following this format `{dir index}_{image index}_{label index}.png`, e.g. `001_02.png`.
It's basically similar to training dataset but there is no label.

In [26]:
def process_dir_test(dirpath: Path):
    with open(dirpath / f"{dirpath.name}.json") as f:
        j = json.load(f)

    diridx = int(dirpath.name)

    for i, annot in enumerate(j["annotations"]):
        impath = dirpath / f"{i}.png"
        im_org, im = process_image(impath, np.array(annot["data"]))

        # save image
        fname = f"{diridx:03d}_{i:02d}"
        imageio.imwrite(test_out_dir / (fname + ".png"), im_org)
        torch.save(im, test_out_dir / (fname + ".pth"))

In [27]:
dirs = sorted(list(Path("./data/ori/test").glob("*")))

In [28]:
len(dirs)  # 217

217

In [29]:
with Pool() as pool:
    with tqdm(total=len(dirs), ncols=100, file=sys.stdout) as t:
        for _ in pool.imap_unordered(process_dir_test, dirs):
            t.update()

100%|█████████████████████████████████████████████████████████████| 217/217 [04:15<00:00,  1.18s/it]
